In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 90% !important; }</style>"))

In [ ]:
from collections import defaultdict

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

In [ ]:
examplewords = ['talo', 'valo', 'voi', 'koira', 'voida']
freqs = { 'talo': 1, 'valo': 5, 'koira': 3, 'voi': 10, 'oiva': 2, 'voida': 1}
ig = defaultdict(int)
fg = defaultdict(int)
bg = defaultdict(int)

In [ ]:
for word in examplewords:
    ig[word[:3]] += freqs[word]
    fg[word[-3:]] += freqs[word]
    for i,j in zip(range(0, len(word)-1), range(2, len(word)+1)):
        fragment = word[i:j]
        bg[fragment] += freqs[word]

igcols = ['word']
fgcols = ['word']
bgcols = ['word']
igcols.extend(ig.keys())
fgcols.extend(fg.keys())
bgcols.extend(bg.keys())

igdf = pd.DataFrame(columns=igcols)
fgdf = pd.DataFrame(columns=fgcols)
bgdf = pd.DataFrame(columns=bgcols)

for word in examplewords:
    row = [word]
    for igk in ig.keys():
        if igk == word[:3]:
            row.append(freqs[word])
        else:
            row.append(0)
    pdf = pd.DataFrame([row], columns=igcols)
    igdf = pd.concat([igdf, pdf], axis=0, ignore_index=True)

    row = [word]
    for fgk in fg.keys():
        if fgk == word[-3:]:
            row.append(freqs[word])
        else:
            row.append(0)
    pdf = pd.DataFrame([row], columns=fgcols)
    fgdf = pd.concat([fgdf, pdf], axis=0, ignore_index=True)

    row = [word]
    # This doesn't account for the possibility that a bigram may appear in a word multiple times.
    for bgk in bg.keys():
        if bgk in word:
            row.append(freqs[word])
        else:
            row.append(0)
    pdf = pd.DataFrame([row], columns=bgcols)
    bgdf = pd.concat([bgdf, pdf], axis=0, ignore_index=True)


In [ ]:
igdf

In [ ]:
fgdf

In [ ]:
bgdf

In [ ]:
for word in examplewords:
    ini = word[:3]
    fin = word[-3:]
    colsum1 = igdf[ini]
    colsum2 = fgdf[fin]
    # print(word, np.sum(colsum1.values), np.sum(colsum2.values))
    print(f"word {word:5s}, initrigram '{ini}': {np.sum(colsum1.values):2d}, fintrigram '{fin}': {np.sum(colsum2.values):2d}")

In [ ]:
for word in examplewords:
    bgs = bgdf[bgdf.word == word]
    bgcols = [ c for c in bgdf.columns[1:] if bgs.loc[:,c].values[0] > 0]
    colsums = [ bgdf[c].sum() for c in bgcols ]
    avg = int(np.average(colsums))
    print(f'word {word:5s}, columns: {",".join(bgcols):12s}: sums {str(colsums):14s} -> average {avg}')